In [0]:
query="""
select palavras_busca
from bronze.tabnews.palavras"""

resultado=spark.sql(query)

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql import Row
from pyspark.sql.functions import collect_set, concat_ws, sort_array
import pandas as pd


palavras_busca = resultado.collect()
resultados = []

for palavra in palavras_busca:
    palavra = palavra[0]
    query = f"""
    SELECT '{palavra}' as palavra_busca, id, slug, title, source_url, created_at, updated_at, owner_username
    FROM bronze.tabnews.assunto
    WHERE title LIKE '%{palavra}%'
    """
    df = spark.sql(query)
    resultados.append(df.collect())

df_palavras = spark.createDataFrame([row for sublist in resultados for row in sublist])
df_agrupado = df_palavras.groupBy("id", "slug", "title", "source_url", "created_at", "updated_at", "owner_username") \
    .agg(sort_array(collect_set("palavra_busca")).alias("palavras_busca_array")) \
    .withColumn("palavra_busca", concat_ws(", ", "palavras_busca_array")) \
    .drop("palavras_busca_array")\
    .withColumn("url", lit(None).cast("string")) \
    .withColumn("body", lit(None).cast("string"))

In [0]:
df_agrupado.write \
        .format('delta') \
        .mode('overwrite') \
        .option("overwriteSchema", "true") \
        .saveAsTable('silver.tabnews.assunto')